# Previsão de Eleição dos Deputados
Este documento tem objetivo de prever se determinado candidato será eleito ou não baseado em eleições passadas, utilizando aprendizagem de máquina para isto.

In [1]:
import pandas as pd
import numpy as np
import matplotlib
from sklearn import preprocessing

Para realizar a modelagem dos algortmos preditivos, vamos utilizar no treino dados referentes às eleições de 2006 e 2010 e para o teste dados das eleições de 2014.

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
pd.set_option('display.max_columns', 500)
train.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao,situacao
0,2006,10001,JOSÉ LUIZ NOGUEIRA DE SOUSA,AP,PT,6,6,16600.00,2766.67,0.0,9000.0,6300.00,1300.0,0.0,14,14,16583.60,1184.54,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),VEREADOR,nao_eleito
1,2006,10002,LOIVA DE OLIVEIRA,RO,PT,13,13,22826.00,1755.85,6625.0,15000.0,1000.00,201.0,0.0,24,23,20325.99,846.92,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),SERVIDOR PÚBLICO ESTADUAL,nao_eleito
2,2006,10002,MARIA DALVA DE SOUZA FIGUEIREDO,AP,PT,17,16,158120.80,9301.22,2250.0,34150.0,62220.80,59500.0,0.0,123,108,146011.70,1187.09,DEPUTADO FEDERAL,FEMININO,SUPERIOR COMPLETO,VIÚVO(A),PEDAGOGO,eleito
3,2006,10002,ROMALDO MILANI,MS,PRONA,6,6,3001.12,500.19,0.0,1150.0,1101.12,750.0,0.0,8,8,3001.12,375.14,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO INCOMPLETO,CASADO(A),MILITAR REFORMADO,nao_eleito
4,2006,10003,ANSELMO DE JESUS ABREU,RO,PT,48,48,119820.00,2496.25,0.0,50878.0,0.00,68942.0,0.0,133,120,116416.64,875.31,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL COMPLETO,CASADO(A),DEPUTADO,eleito


In [4]:
test.head()

,ano,sequencial_candidato,nome,uf,partido,quantidade_doacoes,quantidade_doadores,total_receita,media_receita,recursos_de_outros_candidatos.comites,recursos_de_pessoas_fisicas,recursos_de_pessoas_juridicas,recursos_proprios,recursos_de_partido_politico,quantidade_despesas,quantidade_fornecedores,total_despesa,media_despesa,cargo,sexo,grau,estado_civil,ocupacao
0,2014,10000000135,EMERSON DA SILVA SANTOS,AC,PSOL,3,3,1580.00,526.67,0.00,1500.00,0.00,0.00,80.0,3,3,1580.00,526.67,DEPUTADO FEDERAL,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),"CORRETOR DE IMÓVEIS, SEGUROS, TÍTULOS E VALORES"
1,2014,10000000142,GERALDO SILVA DOS SANTOS,AC,PSOL,5,5,3180.00,636.00,0.00,3100.00,0.00,0.00,80.0,6,5,3130.02,521.67,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),VIGILANTE
2,2014,10000000158,CARLOS CESAR CORREIA DE MESSIAS,AC,PSB,40,38,336793.13,8419.83,1923.07,65700.00,154170.06,115000.00,0.0,145,139,326869.78,2254.27,DEPUTADO FEDERAL,MASCULINO,ENSINO FUNDAMENTAL INCOMPLETO,CASADO(A),OUTROS
3,2014,10000000161,IDESIO LUIS FRANKE,AC,PT,29,29,156719.32,5404.11,39122.32,68783.84,17000.00,6813.16,25000.0,136,121,241016.07,1772.18,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),AGRÔNOMO
4,2014,10000000163,LEONARDO CUNHA DE BRITO,AC,PT,160,146,737073.00,4606.71,10000.00,347073.00,277000.00,78000.00,25000.0,518,354,567401.15,1095.37,DEPUTADO FEDERAL,MASCULINO,SUPERIOR COMPLETO,CASADO(A),ADVOGADO


Como uma análise prévia para estes dados já tinham sido feitas, que podem ser vista neste [notebook](https://github.com/orionwinter/machine-learning/blob/master/regressao-varios-modelos/Previs%C3%A3o%20de%20Vota%C3%A7%C3%A3o%20de%20Deputados.ipynb), os dados numéricos são bastante desbalanceados, além de existirem valores faltantes. Vamos novamente contornar esses detalhes aplicando a função de log nos dados numéricos e substituindo os valores faltantes pela média da coluna. Além disso, os valores numéricos foram normalizados entre 0 e 1, para melhorar o desempenho dos modelos. Para a utilização das variáveis categóricas, vamos criar variáveis dummies, que representam numericamente as classes, exceto para o nome do candidato.

In [5]:
# Remove nome dos candidatos
train = train.drop(["sequencial_candidato", "nome", "cargo"], axis=1)
test = test.drop(["sequencial_candidato", "nome", "cargo"], axis=1)

# Utiliza escala logarítmica
numeric_feats = train.dtypes[train.dtypes != "object"].index
train[numeric_feats] = np.log1p(train[numeric_feats])
test[numeric_feats] = np.log1p(test[numeric_feats])

# Preenche os valores faltantes
train = train.fillna(train.mean())
test = test.fillna(test.mean())

# Normaliza variáveis numéricas entre 0 e 1
numeric_feats = train.dtypes[train.dtypes != "object"].index
train_non_numeric_feats = train.dtypes[train.dtypes == "object"].index
test_non_numeric_feats = test.dtypes[test.dtypes == "object"].index

min_max_scaler = preprocessing.MinMaxScaler()
train_normalized = min_max_scaler.fit_transform(train[numeric_feats])
test_normalized = min_max_scaler.fit_transform(test[numeric_feats])

train_normalized = pd.DataFrame(train_normalized, columns=train[numeric_feats].columns)
test_normalized = pd.DataFrame(test_normalized, columns=test[numeric_feats].columns)

train_normalized = pd.concat([train_normalized, train[train_non_numeric_feats].reset_index()], axis = 1, sort = False)
test_normalized = pd.concat([test_normalized, test[test_non_numeric_feats].reset_index()], axis = 1, sort = False)

train = train_normalized
test = test_normalized

# Cria variáveis dummies
transformed_data = pd.concat([train[train.columns[train.columns != "situacao"]], test], keys=["train", "test"])
train = pd.concat([train[["situacao"]].reset_index(drop=True), pd.get_dummies(transformed_data).loc["train"]], axis=1)
test = pd.get_dummies(transformed_data).loc["test"]

Primeiramente vamos verificar se existe desbalanceamento na variável alvo:

In [6]:
sit = train[["situacao"]].groupby(["situacao"]).size()
sit.plot(kind = "bar")

In [7]:
float(sit[0]) / float(sit[1])

0.15554881746513038

Podemos ver que existem muito mais candidatos não eleitos do que eleitos, em uma proporção de 0.156, que é cerca de 1 eleito para cada 6 não eleitos.

Como alguns algoritmos de classificação são bastante sensíveis ao desbalanceamento, isso poderia tendenciá-los a predizer pela classe majoritária, ou seja, ter uma tendência muito maior a prever que um deputado não foi eleito, visto este desbalanceamento.

Para tratar este caso, vamos utilizar uma técnica de reamostragem chamada SMOTE(Synthetic Minority Over-sampling Technique) que é um algoritmo um pouco mais rebuscado que uma simples reamostragem da classe minoritária, visto que além de reamostrar modifica sinteticamente os dados reamostrados baseado nos demais da mesma classe.

Vale ressaltar que utilizaremos os dados de 2006 para treino e 2010 para validação, logo só balancearemos os dados de treino.

In [8]:
from imblearn.over_sampling import SMOTE

def balanceia_dados(dados):
    sm = SMOTE(random_state=42)
    X = dados.loc[:, dados.columns != "situacao"]
    y = dados.situacao
    X_res, y_res = sm.fit_resample(X, y)
    dados_balanceados = pd.concat([pd.DataFrame(y_res), pd.DataFrame(X_res).reset_index(drop=True)], axis=1)
    return dados_balanceados

train_data = train[train.ano == 0]
validation_data = train[train.ano == 1]
train_bal = balanceia_dados(train_data)
train_bal.columns = train_data.columns

/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/parallel.py:268: DeprecationWarning: check_pickle is deprecated in joblib 0.12 and will be removed in 0.13
  ' removed in 0.13', DeprecationWarning)


## Treinamento de modelos

Agora vamos treinar 4 modelos diferentes: regressão logística, KNN, árvore de decisão e adaboost. Para será utilizado cross validation com 5 folds.

In [43]:
# Regressão logistica
from sklearn.linear_model import LogisticRegressionCV

X = train_bal.loc[:, train_bal.columns != "situacao"]
Y = train_bal.situacao
X_validation = validation_data.drop('situacao', axis=1)
Y_validation = validation_data.situacao

lr_model = LogisticRegressionCV(cv=5, random_state=0,
                                multi_class='multinomial').fit(X, Y)
lr_train_predict = lr_model.predict(train_data.drop('situacao', axis=1))
lr_train_y_score = lr_model.decision_function(train_data.drop('situacao', axis=1))
lr_validation_predict = lr_model.predict(X_validation)
lr_validation_y_score = lr_model.decision_function(X_validation)

Agora vamos reportar o Precision, Recall, AUC da curva ROC e AUC-Precision&Recall no treino e validação para cada modelo treinado.

In [50]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, precision_recall_curve, auc

train_y_true = train_data.situacao.values
validation_y_true = validation_data.situacao.values

lr_train_precision = precision_score(train_y_true, lr_train_predict, pos_label="eleito")
lr_train_recall = recall_score(train_y_true, lr_train_predict, pos_label="eleito")
lr_train_auc_roc = roc_auc_score(train_y_true, lr_train_y_score)

lr_validation_precision = precision_score(validation_y_true, lr_validation_predict, pos_label="eleito")
lr_validation_recall = recall_score(validation_y_true, lr_validation_predict, pos_label="eleito")
lr_validation_auc_roc = roc_auc_score(validation_y_true, lr_validation_y_score)

In [46]:
print (lr_train_precision, lr_train_recall, lr_train_auc_roc)
print (lr_validation_precision, lr_validation_recall, lr_validation_auc_roc, lr_validation_auc)

(0.6216897856242118, 0.9610136452241715, 0.9682200212976305, 0.07658465470855652)
(0.5530474040632054, 0.9551656920077972, 0.9671168662396733, 0.06740167787992918)
